In [1]:
import re
import glob
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np

from nltk import sent_tokenize, word_tokenize, regexp_tokenize
from nltk.corpus import stopwords

from collections import Counter

import requests
from bs4 import BeautifulSoup as BS
from tqdm.notebook import tqdm
import codecs
import html5lib
from IPython.core.display import HTML

from spacy.lang.en import English
import spacy

from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher

from nltk.tokenize import word_tokenize, sent_tokenize
from collections import defaultdict
from spacy import displacy

In [2]:
globs = glob.glob('../data/*.htm') + glob.glob('../data/*.html')

In [21]:
globs[1]

'../data\\cnxc-20211130.htm'

In [13]:
globs[20]
with open('../data/amrk-10k_20210630.htm') as fi:
    file_html = fi.read()

In [16]:
htmlFile = open(globs[1])
soup = BS(htmlFile)
print(soup.body.text[0:10000])

FALSE2021FY0001803599P4Y0.0050.005http://fasb.org/us-gaap/2021-01-31#OtherAssetsNoncurrenthttp://fasb.org/us-gaap/2021-01-31#OtherAssetsNoncurrenthttp://fasb.org/us-gaap/2021-01-31#OtherAssetsNoncurrenthttp://fasb.org/us-gaap/2021-01-31#OtherAccruedLiabilitiesCurrenthttp://fasb.org/us-gaap/2021-01-31#OtherAccruedLiabilitiesCurrenthttp://fasb.org/us-gaap/2021-01-31#OtherAccruedLiabilitiesCurrenthttp://fasb.org/us-gaap/2021-01-31#OtherLiabilitiesNoncurrenthttp://fasb.org/us-gaap/2021-01-31#OtherLiabilitiesNoncurrenthttp://fasb.org/us-gaap/2021-01-31#OtherLiabilitiesNoncurrent00018035992020-12-012021-11-3000018035992021-05-28iso4217:USD00018035992022-01-17xbrli:shares00018035992021-11-3000018035992020-11-30iso4217:USDxbrli:shares00018035992019-12-012020-11-3000018035992018-12-012019-11-300001803599us-gaap:CommonStockMember2018-11-300001803599us-gaap:AdditionalPaidInCapitalMember2018-11-300001803599us-gaap:TreasuryStockMember2018-11-300001803599us-gaap:RetainedEarningsMember2018-11-3000018

In [23]:
def remove_tags(htmlFile):
  
    # parse html content
    soup = BS(htmlFile)
  
    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()
  
    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)

htmlFile = open(globs[0])

text = remove_tags(htmlFile)



sentences = sent_tokenize(text)
repurchase_sent = [x for x in sentences if 'repurchas' in x]
repurchase_sent = [x for x in repurchase_sent if re.search(r'\d', x)]
repurchase_sent = [x.replace('\xa0', " ") for x in repurchase_sent]
repurchase_sent

['Fiscal 2021 Fiscal 2020 Quarter High Low High Low First $ 34.51 $ 18.99 $ 14.47 $ 11.40 Second $ 37.40 $ 25.08 $ 13.26 $ 8.00 Third $ 39.74 $ 26.67 $ 12.28 $ 7.85 Fourth $ 56.35 $ 35.11 $ 19.56 $ 12.13 Issuer Purchases of Equity Securities On April 26, 2018, the Company’s Board of Directors authorized a stock repurchase program for up to 500,000 shares of the Company’s stock.',
 'As of September 3, 2021, there have been no repurchases of equity securities under the above-referenced stock repurchase program.',
 'The overall increase is primarily due to $1.2 million interest earned from repurchase agreements and $0.2 million of interest income earned from spot deferred trade orders.',
 'As of June 30, 2021 and June 30, 2020, included within inventories is $ 75.1 million and $ 71.0 million, respectively, of precious metals products subject to repurchase arrangements with customers.',
 "Interest Income During the years ended June 30, 2021 and 2020, the Company earned interest income rela

In [24]:
nlp = spacy.load('en_core_web_md')

In [28]:
from spacy.matcher import DependencyMatcher
matcher = DependencyMatcher(nlp.vocab)

In [80]:
pattern = [
    {
        "RIGHT_ID": "anchor_authorized",
        "RIGHT_ATTRS":{"LEMMA": 'authorize'}
    },
    {
        'LEFT_ID': 'anchor_authorized',
        'REL_OP': '>>',
        'RIGHT_ID': 'auhorized_subj',
        'RIGHT_ATTRS' : {'DEP' : 'nsubj'}
    },
    {
        'LEFT_ID': "anchor_authorized",
        'REL_OP' : '>>',
        'RIGHT_ID' : 'authorized_prep',
        'RIGHT_ATTRS' : {'DEP' : 'prep'}
    },
    {
        'LEFT_ID': "authorized_prep",
        'REL_OP' : '>>',
        'RIGHT_ID' : 'prep_object',
        'RIGHT_ATTRS' : {'DEP' : 'pobj'}
    },
    {
        'LEFT_ID': "prep_object",
        'REL_OP' : '.',
        'RIGHT_ID' : 'amount_pobj',
        'RIGHT_ATTRS' : {'DEP' : 'nummod'}
    }
]

In [81]:
matcher.add('AUTHORIZED', [pattern])
doc = nlp("On April 26, 2018, the Company’s Board of Directors authorized a stock repurchase program for up to 500,000 shares of the Company’s stock.")

In [82]:
matches = matcher(doc)
print(matches)

[(14036459411071875226, [12, 9, 0, 1, 2]), (14036459411071875226, [12, 9, 0, 1, 2]), (14036459411071875226, [12, 9, 0, 1, 2]), (14036459411071875226, [12, 9, 0, 1, 2]), (14036459411071875226, [12, 9, 17, 21, 20]), (14036459411071875226, [12, 9, 17, 21, 20]), (14036459411071875226, [12, 9, 0, 1, 2]), (14036459411071875226, [12, 9, 0, 1, 2])]


In [84]:
match_id, token_ids = matches[1]
for i in range(len(token_ids)):
    print(pattern[i]["RIGHT_ID"] + ":", doc[token_ids[i]].text)

anchor_authorized: authorized
auhorized_subj: Board
authorized_prep: On
prep_object: April
amount_pobj: 26
